In [33]:
import requests
import json
import pandas as pd 
import numpy as np 
from keys import token, company_id, login, password

## Авторизация и получение данных о пользователе

In [ ]:
url = 'https://api.yclients.com/api/v1/auth'

headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/vnd.yclients.v2+json"
}
payload = {
    "login": login,
    "password": password
}

response = requests.post(url, headers=headers, json=payload)
print(response.status_code)
print(response.json()['data'])

## Информация по компании

In [ ]:
url = f"https://api.yclients.com/api/v1/company/{company_id}"

headers = {
    "Accept": "application/vnd.yclients.v2+json",
    "Authorization": token 
}

response = requests.get(url, headers=headers)
response.json()['data']